# 📧 Bernoulli Naive Bayes — Complete Guide

## 🌟 Real-Life Example: Short Text Classification

Imagine you're building a system to classify **short social media posts** as either **SPAM** or **LEGITIMATE**.

### **Your Training Data:**
| Message | Contains "FREE"? | Contains "WIN"? | Contains "CLICK"? | Contains "MEETING"? | Label |
|---------|------------------|-----------------|-------------------|---------------------|-------|
| "Free win now!" | ✅ Yes | ✅ Yes | ❌ No | ❌ No | SPAM |
| "Click here to win!" | ✅ Yes | ✅ Yes | ✅ Yes | ❌ No | SPAM |
| "Meeting at 3pm" | ❌ No | ❌ No | ❌ No | ✅ Yes | LEGIT |
| "Free meeting today" | ✅ Yes | ❌ No | ❌ No | ✅ Yes | LEGIT |

**Key Point**: We only care if words are **present (1)** or **absent (0)**, not how many times they appear.

---

## 🧠 How Bernoulli Naive Bayes Works

### **The Core Idea**
Instead of counting word frequencies (like Multinomial NB), Bernoulli NB only asks:
> **"Is this word present or absent in the message?"**

Each feature is **binary**: 1 = word present, 0 = word absent.

---

## 📊 Step-by-Step Example

### **Step 1: Your Binary Feature Matrix**
```python
# Features: [FREE, WIN, CLICK, MEETING]
X = [
    [1, 1, 0, 0],  # "Free win now!" → SPAM
    [1, 1, 1, 0],  # "Click here to win!" → SPAM  
    [0, 0, 0, 1],  # "Meeting at 3pm" → LEGIT
    [1, 0, 0, 1]   # "Free meeting today" → LEGIT
]
y = ["spam", "spam", "legit", "legit"]

### **Step 2: Model Learns Probabilities**

For each word, calculate **P(word present | class)**:

| Word | P(present | spam) | P(present | legit) |
|------|-------------------|--------------------|
| FREE | 2/2 = 1.0 | 1/2 = 0.5 |
| WIN | 2/2 = 1.0 | 0/2 = 0.0 |
| CLICK | 1/2 = 0.5 | 0/2 = 0.0 |
| MEETING | 0/2 = 0.0 | 2/2 = 1.0 |

### **Step 3: Handle Word Absence**
Bernoulli NB also considers **absent words**:
- P(word absent | class) = 1 - P(word present | class)

| Word | P(absent | spam) | P(absent | legit) |
|------|------------------|-------------------|
| FREE | 0.0 | 0.5 |
| WIN | 0.0 | 1.0 |
| CLICK | 0.5 | 1.0 |
| MEETING | 1.0 | 0.0 |

---

## 🔍 Prediction: Classify New Message

### **New Message**: "Win meeting now!"
- Contains: WIN ✅, MEETING ✅
- Doesn't contain: FREE ❌, CLICK ❌
- Binary vector: `[0, 1, 0, 1]`

### **Calculate Scores:**

**SPAM Score:**

P(spam) × P(FREE=0 | spam) × P(WIN=1 | spam) × P(CLICK=0 | spam) × P(MEETING=1 | spam)
= 0.5 × 0.0 × 1.0 × 0.5 × 0.0
= 0.0  ← Problem! Zero probability!

**LEGIT Score:**

P(legit) × P(FREE=0 | legit) × P(WIN=1 | legit) × P(CLICK=0 | legit) × P(MEETING=1 | legit)
= 0.5 × 0.5 × 0.0 × 1.0 × 1.0  
= 0.0  ← Also zero!



---

## 🛠️ Handling Zero Probabilities (Smoothing)

### **With Laplace Smoothing (add 1 to all counts):**
- P(WIN=1 | legit) = (0+1)/(2+2) = 1/4 = 0.25
- P(MEETING=1 | spam) = (0+1)/(2+2) = 1/4 = 0.25

**Recalculated SPAM Score:**

0.5 × (0+1)/(2+2) × (2+1)/(2+2) × (1+1)/(2+2) × (0+1)/(2+2)
= 0.5 × 0.25 × 0.75 × 0.5 × 0.25
= 0.0117


**Recalculated LEGIT Score:**

0.5 × (1+1)/(2+2) × (0+1)/(2+2) × (2+1)/(2+2) × (2+1)/(2+2)
= 0.5 × 0.5 × 0.25 × 0.75 × 0.75
= 0.0352



### **Final Decision:**
- **LEGIT Score (0.0352) > SPAM Score (0.0117)**
- **Prediction: LEGITIMATE** ✅

This makes sense because "meeting" is a strong indicator of legitimate messages!

---

## 📊 When to Use Bernoulli Naive Bayes

### ✅ Perfect For:
- **Short text** (tweets, SMS, headlines)
- **Binary features** (word present/absent)
- **When word frequency doesn't matter** (only presence/absence)
- **Document classification** with binary word occurrence

### ❌ Avoid When:
- **Long documents** (use Multinomial NB instead)
- **Word frequency matters** (e.g., "free free free" vs "free")
- **Continuous features** (use Gaussian NB)

---

## 💡 Key Differences Between Naive Bayes Types

| Type | Feature Type | Use Case |
|------|--------------|----------|
| **Bernoulli** | Binary (0/1) | Short text, word presence/absence |
| **Multinomial** | Counts (0,1,2,3...) | Long text, word frequencies |
| **Gaussian** | Continuous (any number) | Numerical data like height, weight, test scores |

---

## 🎯 Key Takeaways

- **Bernoulli Naive Bayes = Binary presence/absence model**
- **Perfect for short text** where only word existence matters
- **Considers both present AND absent words** in calculations
- **Still "naive"** (assumes word independence)
- **Requires smoothing** to handle zero probabilities
- **Fast and effective** for binary feature problems

> **Remember**: Bernoulli Naive Bayes is like a spam filter that only asks "Does this email contain the word 'FREE'?" rather than "How many times does 'FREE' appear?" 📧

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [16]:
# sample dataset

texts = [
    "Free win now!",
    "Click here to win!",
    "Meeting at 3pm",
    "Free meeting today",
    "Win big prizes!",
    "Schedule meeting soon",
    "Get free rewards now",
    "Project meeting at office",
]

labels = ["spam", "spam", "legit", "legit", "spam", "legit", "spam", "legit"]

In [17]:
vector = CountVectorizer(binary=True)

X = vector.fit_transform(texts)
y = labels

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
model = BernoulliNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [20]:
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print(f"Confusion Matrix:\n {cm}")
print(f"\nClassification Report:\n {cr}")
print(f"Accuracy: {acc}")

Confusion Matrix:
 [[1 0]
 [0 1]]

Classification Report:
               precision    recall  f1-score   support

       legit       1.00      1.00      1.00         1
        spam       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Accuracy: 1.0


#### Testing our model by giving a custom input


In [21]:
new_messages = [
    "Win free cash now!",
    "Meeting scheduled tomorrow",
    "Click here to get offer",
]

X_new = vector.transform(new_messages)
print(f"Predictions on new messege:\n {model.predict(X_new)}")

Predictions on new messege:
 ['spam' 'legit' 'spam']
